In [26]:
#Récupérer les données 

label = np.load("Label.npy")
targetBac = np.load("targetBac.npy")
targetK7 =  np.load("targetK7.npy")


print(len(label))
print(type(label[0][0]))

13423
<class 'numpy.int32'>


In [27]:
#Paramètres

input_size= len(label[0])
hidden_dim = 1000
n_layers = 10
output_size = len(label[0])

batch_size = 200
num_epochs = 1000
n_iter = num_epochs * (len(targetBac) / batch_size)
n_iter = int(n_iter)


In [28]:
#Prep input bac
input_seq = []
targetBac_seq = []
for i in range(len(label)-1):
    #input_seq.append([label[i],targetBac[i]])
    #targetBac_seq.append([label[i+1],targetBac[i+1]])
    input_seq.append(np.concatenate((label[i],targetBac[i]),axis=0))
    targetBac_seq.append(np.concatenate((label[i+1],targetBac[i+1]),axis=0))
    
input_seq = torch.from_numpy(np.asarray(input_seq))
targetBac_seq = torch.from_numpy(np.asarray(targetBac_seq))


#Adding an axis to input 
input_seq = input_seq[None,:,:]
targetBac_seq = targetBac_seq[None,:,:]

input_seq  = input_seq.float()
targetBac_seq = targetBac_seq.float()


print(input_seq.size())

torch.Size([1, 13422, 264])


In [35]:
def create_inout_sequences(input_seq, targetBac_seq):
    inout_seq = []
    for i in range(len(input_seq)):
        train_seq = input_seq[i]
        train_label = targetBac_seq[i]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [36]:
train_inout_seq = create_inout_sequences(input_seq, targetBac_seq)
print(train_inout_seq)

[(tensor([[2.0180e+03, 8.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.0180e+03, 8.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.0180e+03, 8.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [2.0190e+03, 3.0000e+00, 3.0000e+01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.0190e+03, 3.0000e+00, 3.0000e+01,  ..., 0.0000e+00, 1.0000e+00,
         0.0000e+00],
        [2.0190e+03, 3.0000e+00, 3.0000e+01,  ..., 0.0000e+00, 1.1000e+01,
         0.0000e+00]]), tensor([[2.0180e+03, 8.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.0180e+03, 8.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.0180e+03, 8.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [2.0190e+03, 3.0000e+00, 3.0000e+01,  ..., 0.0000e+00, 1.0000e+00,
         0.0000e+00],


In [37]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [38]:
class LSTM(nn.Module):
    def __init__(self, input_size, output_size, n_layers):
        super().__init__()
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, n_layers)

        self.linear = nn.Linear(n_layers, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.n_layers),
                            torch.zeros(1,1,self.n_layers))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [39]:
model = LSTM(input_size=input_seq.size(2) , output_size=targetBac_seq.size(2), n_layers=n_layers)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)


# Define Loss, Optimizer
lr=0.001
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

print(model)

LSTM(
  (lstm): LSTM(264, 10)
  (linear): Linear(in_features=10, out_features=264, bias=True)
)


In [40]:
# Training Run
epochs = 150

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.n_layers),
                        torch.zeros(1, 1, model.n_layers))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        print("Coucou")

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 18006.66796875
epoch:  26 loss: 18004.52343750


KeyboardInterrupt: 

In [2]:
print(targetBac_seq.size())

NameError: name 'targetBac_seq' is not defined

In [171]:
print(input_seq.size())

torch.Size([1, 13422, 264])


In [131]:
print(targetBac_seq.long())

tensor([[[2018,    8,    1,  ...,    0,    0,    0],
         [2018,    8,    1,  ...,    0,    0,    0],
         [2018,    8,    1,  ...,    0,    0,    0],
         ...,
         [2019,    3,   30,  ...,    0,    1,    0],
         [2019,    3,   30,  ...,    0,   11,    0],
         [2019,    3,   30,  ...,    0,    0,    0]]])
